In [1]:
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
pd.options.display.float_format = '{:,.0f}'.format

In [3]:
def 사업실적표_불러오기(기준년월, 파일경로='./data/'):
    파일경로_수정 = 파일경로
    if(파일경로[-1] not in ['\\', '/']):
        파일경로_수정 = 파일경로_수정+'/'
    사업실적표 = pd.read_excel('{}업무보고서_{}.xlsx'.format(파일경로_수정, 기준년월), sheet_name='AI059')
    return 사업실적표

In [4]:
def 사업실적표_가공하기(사업실적표):
    #1. 데이터 가공
    사업실적표_인덱스 = 사업실적표.iloc[13:, 0].str.replace('\r\n', '').reset_index(drop=True)
    사업실적표_인덱스.name = None

    사업실적표_컬럼 = 사업실적표.iloc[9,3:].reset_index(drop=True)
    사업실적표_컬럼.name = None

    사업실적표_가공 = 사업실적표.iloc[13:, 3:].astype(float).fillna(0)
    사업실적표_가공.index = 사업실적표_인덱스
    사업실적표_가공.columns = 사업실적표_컬럼
    
    return 사업실적표_가공

In [5]:
def 지급보험금보험환급금_식별하기(사업실적표_가공):
    #행, 열 키 식별
    일반 = ['A11', 'A12', 'A14', 'A151', 'A156', 'A152', 'A153', 'A157', 'A154', 'A155', 'A18', 'A16', 'A17', 'A4']
    자동차 = ['A13']
    장기 = ['A21', 'A22', 'A23', 'A29', 'B']
    지급보험금 = ['F1', 'F2']
    보험환급금 = ['F4', 'F5', 'F12']
    
    #값 계산
    지급보험금_보험환급금_일반 = 사업실적표_가공.loc[일반, 지급보험금+보험환급금].sum()
    지급보험금_일반 = 지급보험금_보험환급금_일반['F1'] - 지급보험금_보험환급금_일반['F2']
    보험환급금_일반 = 지급보험금_보험환급금_일반[보험환급금].sum()
    지급보험금_보험환급금_장기 = 사업실적표_가공.loc[장기, 지급보험금+보험환급금].sum()
    지급보험금_장기 = 지급보험금_보험환급금_장기['F1'] - 지급보험금_보험환급금_장기['F2']
    보험환급금_장기 = 지급보험금_보험환급금_장기[보험환급금].sum()
    지급보험금_보험환급금_자동차 = 사업실적표_가공.loc[자동차, 지급보험금+보험환급금].sum()
    지급보험금_자동차 = 지급보험금_보험환급금_자동차['F1'] - 지급보험금_보험환급금_자동차['F2']
    보험환급금_자동차 = 지급보험금_보험환급금_자동차[보험환급금].sum()
    보험환급금_전체 = 보험환급금_일반 + 보험환급금_장기 + 보험환급금_자동차
    지급보험금_전체 = 지급보험금_일반 + 지급보험금_장기 + 지급보험금_자동차
    
    # 집계
    지급보험금_보험환급금_집계 = pd.Series([보험환급금_일반, 보험환급금_장기, 보험환급금_자동차, 보험환급금_전체, 지급보험금_일반, 지급보험금_장기, 지급보험금_자동차, 지급보험금_전체], index=['보험환급금_일반', '보험환급금_장기', '보험환급금_자동차', '보험환급금_전체', '지급보험금_일반', '지급보험금_장기', '지급보험금_자동차', '지급보험금_전체']).to_frame().T
    return 지급보험금_보험환급금_집계

In [6]:
def 기준년월_생성하기(시작년월, 종료년월, 주기):
    시작 = datetime(int(시작년월[:4]), int(시작년월[-2:]), 1)
    종료 = datetime(int(종료년월[:4]), int(종료년월[-2:]), 1)
    기준년월 = 시작
    기준년월_전체 = []
    while(기준년월 <= 종료):
        기준년월_전체.append(기준년월.strftime('%Y%m'))
        기준년월 = 기준년월 + relativedelta(months=주기)
    
    return 기준년월_전체

In [7]:
def 지급보험금보험환급금_생성하기(시작년월, 종료년월, 주기=1):
    기준년월_전체 = 기준년월_생성하기(시작년월, 종료년월, 주기)

    지급보험금_보험환급금_누적 = []
    for 기준년월 in 기준년월_전체:
        사업실적표 = 사업실적표_불러오기(기준년월)
        사업실적표_가공 = 사업실적표_가공하기(사업실적표)
        지급보험금_보험환급금 = 지급보험금보험환급금_식별하기(사업실적표_가공)
        지급보험금_보험환급금_수정 = 지급보험금_보험환급금.copy()
        지급보험금_보험환급금_수정.index = [기준년월]
        지급보험금_보험환급금_누적.append(지급보험금_보험환급금_수정)
    지급보험금_보험환급금_누적 = pd.concat(지급보험금_보험환급금_누적)

    지급보험금_보험환급금_당월 = 지급보험금_보험환급금_누적.loc[기준년월_전체[1:]]

    for i in range(1, len(기준년월_전체)):
        기준년월_당월 = 기준년월_전체[i]
        기준년월_전월 = 기준년월_전체[i-1]
        if int(기준년월_당월[-2:]) == 주기:
            continue
        지급보험금_보험환급금_당월.loc[기준년월_당월] = 지급보험금_보험환급금_누적.loc[기준년월_당월] - 지급보험금_보험환급금_누적.loc[기준년월_전월]
        
    return 지급보험금_보험환급금_누적, 지급보험금_보험환급금_당월

In [ ]:
if __name__ == '__main__':
    # 기준설정
    시작년월 = '201812'
    종료년월 = '201903'
    주기 = 1
    
    # 작업
    지급보험금_보험환급금_누적, 지급보험금_보험환급금_당월 = 지급보험금보험환급금_생성하기(시작년월, 종료년월, 주기)
    
    # 결과 출력
    writer = pd.ExcelWriter('./result/지급보험금_보험환급금_{}_{}_{}_{}.xlsx'.format(시작년월, 종료년월, 주기, datetime.now().strftime('%Y%m%d%H%M%S')), 'xlsxwriter')
    지급보험금_보험환급금_누적.to_excel(writer, '누적')
    지급보험금_보험환급금_당월.to_excel(writer, '당월')
    writer.save()
    writer.close()

In [ ]:
지급보험금_보험환급금_누적

In [ ]:
지급보험금_보험환급금_당월